In [1]:
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
stop_words = stopwords.words("english")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import nltk

In [2]:
data = pd.read_csv("imdb_reviews.csv")

In [3]:
data = data.drop("id",axis = 1)

In [4]:
data

,text,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures movies is not a fitting word fo...,neg
...,...,...
49454,"Seeing as the vote average was pretty low, and...",pos
49455,"The plot had some wretched, unbelievable twist...",pos
49456,I am amazed at how this movieand most others h...,pos
49457,A Christmas Together actually came before my t...,pos


In [5]:
data['sentiment'].value_counts()

neg    24765
pos    24694
Name: sentiment, dtype: int64

In [6]:
# ---------- number of words counts passing in the first text 
countvec = CountVectorizer(analyzer = 'word')
count1 = countvec.fit(data.iloc[0])
print("Print Vocabulary :" + str(count1.vocabulary_)+ '\n\n')

Print Vocabulary :{'once': 74, 'again': 1, 'mr': 64, 'costner': 24, 'has': 42, 'dragged': 29, 'out': 77, 'movie': 63, 'for': 36, 'far': 33, 'longer': 60, 'than': 95, 'necessary': 66, 'aside': 12, 'from': 39, 'the': 96, 'terrific': 94, 'sea': 87, 'rescue': 86, 'sequences': 88, 'of': 71, 'which': 113, 'there': 98, 'are': 8, 'very': 107, 'few': 34, 'just': 54, 'did': 27, 'not': 69, 'care': 17, 'about': 0, 'any': 5, 'characters': 19, 'most': 62, 'us': 106, 'have': 43, 'ghosts': 40, 'in': 50, 'closet': 20, 'and': 4, 'costners': 25, 'character': 18, 'realized': 84, 'early': 31, 'on': 73, 'then': 97, 'forgotten': 38, 'until': 105, 'much': 65, 'later': 59, 'by': 16, 'time': 101, 'we': 110, 'should': 89, 'really': 85, 'is': 52, 'cocky': 22, 'overconfident': 79, 'ashton': 11, 'kutcher': 57, 'problem': 83, 'he': 44, 'comes': 23, 'off': 72, 'as': 10, 'kid': 56, 'who': 114, 'thinks': 99, 'hes': 46, 'better': 15, 'anyone': 6, 'else': 32, 'around': 9, 'him': 47, 'shows': 90, 'no': 68, 'signs': 91, 'c

In [7]:
#-------------------------- Stemming
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
my_lines_list = data["text"]

porter=PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

print(my_lines_list[0])
print("STEMMED SENTENCE: ")
x=stemSentence(my_lines_list[0])
print(x)

Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costners character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks hes better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutchers ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.
STEMMED SENTENCE: 
onc again mr. costner ha drag out a movi for far longer than necessari . asid from 

In [8]:
#--------------------------- Lemmatization
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = data["text"]
punctuations="?:!.,;"
sentence_words = nltk.word_tokenize(sentence[0])
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word)))

Word                Lemma               
Once                Once                
again               again               
Mr.                 Mr.                 
Costner             Costner             
has                 ha                  
dragged             dragged             
out                 out                 
a                   a                   
movie               movie               
for                 for                 
far                 far                 
longer              longer              
than                than                
necessary           necessary           
Aside               Aside               
from                from                
the                 the                 
terrific            terrific            
sea                 sea                 
rescue              rescue              
sequences           sequence            
of                  of                  
which               which               
there           

In [9]:
#---------------------- Count Vectorizer
vectorizer = CountVectorizer(analyzer = 'word',max_features=10)
count_model = vectorizer.fit(data['text'])
X = count_model.transform(data['text'])
count_model.get_feature_names()

['and', 'in', 'is', 'it', 'of', 'that', 'the', 'this', 'to', 'was']

In [10]:
X.todense()[:5]

matrix([[ 3,  2,  3,  2,  5,  0,  7,  1,  3,  1],
        [ 9,  2,  3,  1,  5,  2, 14,  7,  3,  3],
        [12,  2,  9,  1,  3,  1, 12,  2,  1,  0],
        [15,  7,  4,  5,  5,  1, 13,  7,  7,  3],
        [ 8,  4,  5,  2,  8,  0, 15,  3,  3,  0]], dtype=int64)

In [11]:
vectorizer = CountVectorizer(analyzer = 'word',max_features=10,max_df=0.3)
count_model = vectorizer.fit(data['text'])
X = count_model.transform(data['text'])
count_model.get_feature_names()

['had',
 'her',
 'me',
 'really',
 'she',
 'story',
 'their',
 'well',
 'were',
 'which']

In [12]:
X.todense()[:5]

matrix([[0, 0, 0, 1, 0, 0, 0, 1, 0, 2],
        [0, 0, 0, 2, 0, 0, 0, 1, 0, 3],
        [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 2, 0],
        [0, 2, 0, 3, 3, 1, 2, 1, 0, 0]], dtype=int64)

In [14]:
# ------------ Tfidf Vectorizer 
vectorizer_tfidf = TfidfVectorizer(analyzer='word',max_features=10,max_df=0.3)
tfidf_model = vectorizer_tfidf.fit(data['text'])
X_tfidf = tfidf_model.transform(data['text'])
tfidf_model.get_feature_names()

['had',
 'her',
 'me',
 'really',
 'she',
 'story',
 'their',
 'well',
 'were',
 'which']

In [15]:
X_tfidf.todense()[:5]

matrix([[0.        , 0.        , 0.        , 0.40585313, 0.        ,
         0.        , 0.        , 0.405903  , 0.        , 0.81885652],
        [0.        , 0.        , 0.        , 0.53150329, 0.        ,
         0.        , 0.        , 0.2657843 , 0.        , 0.80427791],
        [0.56860258, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.58421378, 0.        , 0.57912466, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.41338428, 0.39676709, 0.81956654, 0.        ],
        [0.        , 0.37944341, 0.        , 0.52408471, 0.62234036,
         0.17600976, 0.3640675 , 0.17471637, 0.        , 0.        ]])

In [16]:
#------------------------ Creating Logistic Regression Full Model
vectorizer_logtfidf = TfidfVectorizer(analyzer='word',max_features=100000)
tfidf_model = vectorizer_logtfidf.fit(data['text'])
X_tfidf = tfidf_model.transform(data['text'])
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,data['sentiment'],test_size = 0.2,random_state = 0)
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train,y_train)
predLogistic = clf.predict(X_test)
print(classification_report(y_test,predLogistic))
print(confusion_matrix(y_test,predLogistic))

              precision    recall  f1-score   support

         neg       0.91      0.89      0.90      4890
         pos       0.90      0.91      0.90      5002

    accuracy                           0.90      9892
   macro avg       0.90      0.90      0.90      9892
weighted avg       0.90      0.90      0.90      9892

[[4360  530]
 [ 449 4553]]


In [17]:
#------------------------ Creating Logistic Regression Full Model with 1000 features
vectorizer_logtfidf = TfidfVectorizer(analyzer='word',max_features=1000,max_df=0.8)
tfidf_model = vectorizer_logtfidf.fit(data['text'])
X_tfidf = tfidf_model.transform(data['text'])
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,data['sentiment'],test_size = 0.2,random_state = 0)
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train,y_train)
predLogistic = clf.predict(X_test)
print(classification_report(y_test,predLogistic))
print(confusion_matrix(y_test,predLogistic))

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      4890
         pos       0.87      0.88      0.87      5002

    accuracy                           0.87      9892
   macro avg       0.87      0.87      0.87      9892
weighted avg       0.87      0.87      0.87      9892

[[4230  660]
 [ 611 4391]]


In [18]:
#---------------------- Creating Random Forest Classifier Model
from sklearn.ensemble import RandomForestClassifier
vectorizer_logtfidf = TfidfVectorizer(analyzer='word',max_features=1000,max_df=0.2)
tfidf_model = vectorizer_logtfidf.fit(data['text'])
X_tfidf = tfidf_model.transform(data['text'])
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,data['sentiment'],test_size = 0.2,random_state = 0)
clf = OneVsRestClassifier(RandomForestClassifier())
clf.fit(X_train,y_train)
predRF = clf.predict(X_test)
print(classification_report(y_test,predRF))
print(confusion_matrix(y_test,predRF))

              precision    recall  f1-score   support

         neg       0.81      0.84      0.83      4890
         pos       0.84      0.81      0.83      5002

    accuracy                           0.83      9892
   macro avg       0.83      0.83      0.83      9892
weighted avg       0.83      0.83      0.83      9892

[[4115  775]
 [ 940 4062]]


In [19]:
#---------------------- Creating Random Forest Classifier Model with stopwords
from sklearn.ensemble import RandomForestClassifier
vectorizer_logtfidf = TfidfVectorizer(analyzer='word',max_features=1000,max_df=0.8)
tfidf_model = vectorizer_logtfidf.fit(data['text'])
X_tfidf = tfidf_model.transform(data['text'])
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,data['sentiment'],test_size = 0.2,random_state = 0)
clf = OneVsRestClassifier(RandomForestClassifier())
clf.fit(X_train,y_train)
predRF = clf.predict(X_test)
print(classification_report(y_test,predRF))
print(confusion_matrix(y_test,predRF))

              precision    recall  f1-score   support

         neg       0.82      0.84      0.83      4890
         pos       0.84      0.82      0.83      5002

    accuracy                           0.83      9892
   macro avg       0.83      0.83      0.83      9892
weighted avg       0.83      0.83      0.83      9892

[[4115  775]
 [ 881 4121]]


In [20]:
# ---------------------- Creating Multinominal Naive Bayes Algorithm
from sklearn.naive_bayes import MultinomialNB
vectorizer_logtfidf = TfidfVectorizer(analyzer='word',max_features=1000000,max_df=0.8)
tfidf_model = vectorizer_logtfidf.fit(data['text'])
X_tfidf = tfidf_model.transform(data['text'])
X_train,X_test,y_train,y_test = train_test_split(X_tfidf,data['sentiment'],test_size = 0.2,random_state = 0)
clf = OneVsRestClassifier(MultinomialNB())
clf.fit(X_train,y_train)
predRF = clf.predict(X_test)
print(classification_report(y_test,predRF))
print(confusion_matrix(y_test,predRF))

              precision    recall  f1-score   support

         neg       0.84      0.90      0.87      4890
         pos       0.90      0.83      0.86      5002

    accuracy                           0.87      9892
   macro avg       0.87      0.87      0.86      9892
weighted avg       0.87      0.87      0.86      9892

[[4405  485]
 [ 850 4152]]
